In [1]:
import pandas as pd
import scipy
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import theano.tensor as tt
from patsy import dmatrix
sns.set(style="darkgrid")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [18]:
# create batting average dataset (sum over years)
batting = pd.read_csv("~/baseball/core/Batting.csv")
pitching = pd.read_csv("~/baseball/core/Pitching.csv")
# create batting average dataset (sum over years)
batting = pd.read_csv("~/baseball/core/Batting.csv")
pitching = pd.read_csv("~/baseball/core/Pitching.csv")
pitching = pitching.groupby("playerID").agg({"G": "sum"}).reset_index()
pitching = pitching.loc[pitching["G"]>3]
# filter pitchers with 4+ games
batting_pitchers = batting.playerID.isin(pitching.playerID)
batting = batting[~batting_pitchers]
# sum over seasons
batting = batting.groupby("playerID").agg({"AB": "sum", "H": "sum", "yearID":"mean"}).reset_index()
batting = batting.loc[batting["AB"] > 0]
batting["average"] = batting["H"]/batting["AB"]
# add actual player name
people = pd.read_csv("~/baseball/core/People.csv")
people["name"] = people["nameFirst"] + " " + people["nameLast"]
batting = batting.merge(people[["playerID", "name", "bats"]], on="playerID")
# I am using the book values here, not the values that we got in our last notebook. I think the difference comes
# from the fact that we are using more data (data from more recent seasons).
alpha_0 = 101.4
beta_0 = 287.3
# add empirical Bayes estimate
# this is called a point estimate
batting["eb_estimate"] = (batting["H"] + alpha_0) / (batting["AB"] + alpha_0 + beta_0)
# sort by eb_estimate
batting = batting.sort_values("eb_estimate", ascending=False)

In [19]:
batting.head()

,playerID,AB,H,yearID,average,name,bats,eb_estimate
1791,cobbty01,11435,4189,1916.500000,0.366331,Ty Cobb,L,0.362864
4512,hornsro01,8173,2930,1926.291667,0.358497,Rogers Hornsby,R,0.354065
4720,jacksjo01,4981,1772,1914.071429,0.355752,Shoeless Joe Jackson,L,0.348884
465,barnero01,2391,860,1875.333333,0.359682,Ross Barnes,R,0.345865
2377,delahed01,7510,2597,1895.500000,0.345806,Ed Delahanty,R,0.341626


Use MCMC values for our beta binomial model:

```
Mu: 0.14255250930015972
Sigma: 0.001828644970651173
mu_ab: 0.015265514081094674
```

In [20]:
mu = 0.14255250930015972
sigma = 0.001828644970651173
mu_ab = 0.015265514081094674

In [21]:
alpha_0 = mu/sigma
alpha_ab = mu_ab/sigma

In [22]:
batting["alpha_0"] = alpha_0 + alpha_ab * np.log(batting["AB"])
batting["beta_0"]= 1/sigma - batting["alpha_0"]
batting["alpha_1"] = batting["alpha_0"] + batting["H"]
batting["beta_1"] = batting["beta_0"] + batting["AB"] - batting["H"]
batting["new_eb"] = batting["alpha_1"] / (batting["alpha_1"] + batting["beta_1"])

In [23]:
batting.head()

,playerID,AB,H,yearID,average,name,bats,eb_estimate,alpha_0,beta_0,alpha_1,beta_1,new_eb
1791,cobbty01,11435,4189,1916.500000,0.366331,Ty Cobb,L,0.362864,155.962532,390.890474,4344.962532,7636.890474,0.362629
4512,hornsro01,8173,2930,1926.291667,0.358497,Rogers Hornsby,R,0.354065,153.158919,393.694088,3083.158919,5636.694088,0.353579
4720,jacksjo01,4981,1772,1914.071429,0.355752,Shoeless Joe Jackson,L,0.348884,149.024948,397.828058,1921.024948,3606.828058,0.347517
465,barnero01,2391,860,1875.333333,0.359682,Ross Barnes,R,0.345865,142.898198,403.954808,1002.898198,1934.954808,0.341371
2377,delahed01,7510,2597,1895.500000,0.345806,Ed Delahanty,R,0.341626,152.452674,394.400332,2749.452674,5307.400332,0.341256


In [9]:
batting.bats.value_counts()

R    6103
L    3004
B     859
Name: bats, dtype: int64

We have some null values for handedness:

In [24]:
batting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10800 entries, 1791 to 683
Data columns (total 13 columns):
playerID       10800 non-null object
AB             10800 non-null int64
H              10800 non-null int64
yearID         10800 non-null float64
average        10800 non-null float64
name           10764 non-null object
bats           9966 non-null object
eb_estimate    10800 non-null float64
alpha_0        10800 non-null float64
beta_0         10800 non-null float64
alpha_1        10800 non-null float64
beta_1         10800 non-null float64
new_eb         10800 non-null float64
dtypes: float64(8), int64(2), object(3)
memory usage: 1.2+ MB


Use righthanded as default value:

In [25]:
batting["bats"] = batting["bats"].fillna("R")

One hot encode and drop indicator for right handed players:

In [40]:
batting = pd.get_dummies(batting, columns=['bats'])

Use `R` as the default value, hence we can drop that column:

In [41]:
batting.drop("bats_R", inplace=True, axis=1)

Use good old `patsy` to create a spline and do some renaming:

In [ ]:
transformed_batting = dmatrix("cr(train['yearID'],df = 3)", {"train": batting[["yearID"]]}, 
                              return_type='dataframe')
transformed_batting.rename(columns={"cr(train['yearID'], df=3)[0]": "year_1",
                                    "cr(train['yearID'], df=3)[1]": "year_2",
                                    "cr(train['yearID'], df=3)[2]": "year_3"}, inplace=True)
full_batting = batting.merge(transformed_batting[["year_1", "year_2", "year_3"]], left_index=True, right_index=True)

Add some interactions using `patsy` again:

In [56]:
trans_batting = dmatrix("0+(year_1 + year_2 + year_3) * (bats_L + bats_B)", 
                        full_batting, return_type='dataframe')
new_batting = batting.merge(trans_batting[['year_1', 'year_2', 'year_3', 'year_1:bats_L',
       'year_1:bats_B', 'year_2:bats_L', 'year_2:bats_B', 'year_3:bats_L',
       'year_3:bats_B']], left_index=True, right_index=True)

Let's see what we have:

In [63]:
new_batting.head()

,playerID,AB,H,yearID,average,name,eb_estimate,alpha_0,beta_0,alpha_1,beta_1,new_eb,bats_B,bats_L,year_1,year_2,year_3,year_1:bats_L,year_1:bats_B,year_2:bats_L,year_2:bats_B,year_3:bats_L,year_3:bats_B
1791,cobbty01,11435,4189,1916.500000,0.366331,Ty Cobb,0.362864,155.962532,390.890474,4344.962532,7636.890474,0.362629,0,1,0.385753,0.805572,-0.191325,0.385753,0.0,0.805572,0.0,-0.191325,-0.0
4512,hornsro01,8173,2930,1926.291667,0.358497,Rogers Hornsby,0.354065,153.158919,393.694088,3083.158919,5636.694088,0.353579,0,0,0.275787,0.920190,-0.195976,0.000000,0.0,0.000000,0.0,-0.000000,-0.0
4720,jacksjo01,4981,1772,1914.071429,0.355752,Shoeless Joe Jackson,0.348884,149.024948,397.828058,1921.024948,3606.828058,0.347517,0,1,0.414766,0.772548,-0.187314,0.414766,0.0,0.772548,0.0,-0.187314,-0.0
465,barnero01,2391,860,1875.333333,0.359682,Ross Barnes,0.345865,142.898198,403.954808,1002.898198,1934.954808,0.341371,0,0,0.937880,0.086291,-0.024171,0.000000,0.0,0.000000,0.0,-0.000000,-0.0
2377,delahed01,7510,2597,1895.500000,0.345806,Ed Delahanty,0.341626,152.452674,394.400332,2749.452674,5307.400332,0.341256,0,0,0.654586,0.472539,-0.127126,0.000000,0.0,0.000000,0.0,-0.000000,-0.0


In [65]:
# creating named arrays
year_1 = np.array(new_batting["year_1"])
year_2 = np.array(new_batting["year_2"])
year_3 = np.array(new_batting["year_3"])
y1l = np.array(new_batting["year_1:bats_L"])
y1b = np.array(new_batting["year_1:bats_B"])
y2l = np.array(new_batting["year_2:bats_L"])
y2b = np.array(new_batting["year_2:bats_B"])
y3l = np.array(new_batting["year_3:bats_L"])
y3b = np.array(new_batting["year_3:bats_B"])

In [66]:
# taken from https://stackoverflow.com/questions/54505173/finding-alpha-and-beta-of-beta-binomial-distribution-with-scipy-optimize-and-log
from scipy.special import gammaln, logit, digamma

def loglike_betabinom(params, *args):

    k = args[0] # the OVERALL conversions
    n = args[1] # the number of at-bats (AE)
    b = args[2]
    l = args[3]
    y1 = args[4]
    y2 = args[5]
    y3 = args[6]
    mu_y1 = params[5]
    mu_y2 = params[6]
    mu_y3 = params[7]
    mu_y1b = params[8]
    mu_y1l = params[9]
    mu_y2b = params[10]
    mu_y2l = params[11]
    mu_y3b = params[12]
    mu_y3l = params[13]

    alpha = (params[0] + params[1] * np.log(n) + params[3] * b + params[4] * l
             + mu_y1 * y1 + mu_y2 * y2 + mu_y1 * y3 + mu_y1b * b * y1 + mu_y1l * l * y1
             + mu_y2b * b * y2 + mu_y2l * l * y2 + mu_y3b * b * y3 + mu_y3l * l * y3) / params[2], 
    beta = 1/params[2] - alpha

    logpdf = gammaln(n+1) + gammaln(k+alpha) + gammaln(n-k+beta) + gammaln(alpha+beta) - \
     (gammaln(k+1) + gammaln(n-k+1) + gammaln(alpha) + gammaln(beta) + gammaln(n+alpha+beta))
    
    #return -np.sum(logpdf) 
    mask = np.isfinite(logpdf)
    nll = -logpdf[mask].sum()
    return nll

In [69]:
# the trick here is to set a non-zero lower bound for sigma and mu_ab
from scipy.optimize import minimize

init_params = [0.2, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,]
res = minimize(loglike_betabinom, x0=init_params,
            args=(np.array(new_batting['H']), np.array(new_batting['AB']), np.array(new_batting["bats_B"]), np.array(new_batting["bats_L"]),
                  year_1, year_2, year_3),
            method='L-BFGS-B', options={'disp': True, 'maxiter': 500},
              bounds=[(0.001, 0.4), (0.0001, 0.5), (0.001, 0.5), (-0.1, 0.1), (-0.1, 0.1),
                      (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1),
                      (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1), (-0.1, 0.1),])
print(res)

      fun: 37795.43338073921
 hess_inv: <14x14 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 21.27781045, 134.18029994, 117.78174667, -23.89206202,
        28.14922482,   0.54715201,  20.78959369,   0.        ,
        -1.22745405,  -1.38679752, -12.26944732,  25.2854079 ,
       -10.35805326,   4.29354259])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1695
      nit: 95
   status: 0
  success: True
        x: array([ 0.15414599,  0.01524164,  0.00162399,  0.00488026,  0.01199432,
       -0.01458635, -0.01402161,  0.01      ,  0.01019966,  0.01746908,
       -0.01034605, -0.00333706, -0.00384607, -0.01101001])


In [71]:
(mu_0, mu_ab, sigma, mu_b, mu_l,
 mu_y1,mu_y2, mu_y3, mu_y1b, mu_y1l, mu_y2b,
 mu_y2l, mu_y3b, mu_y3l
) = res.x

In [84]:
new_batting["alpha"] = (mu_0 + mu_ab * np.log(new_batting["AB"]) + mu_b * new_batting["bats_B"] + mu_l * new_batting["bats_L"]
             + mu_y1 * year_1 + mu_y2 * year_2 + mu_y1 * year_3 + mu_y1b * new_batting["bats_B"] * year_1 + mu_y1l * new_batting["bats_L"] * year_1
             + mu_y2b * new_batting["bats_B"] * year_2 + mu_y2l * new_batting["bats_L"] * year_2 + mu_y3b * new_batting["bats_B"] * year_3
                        + mu_y3l * new_batting["bats_L"] * year_3) / sigma
new_batting["beta"] = 1/sigma - new_batting["alpha"]

Now, let's calculate the new posterior batting averages:

In [87]:
new_batting["beta_2"] = new_batting["beta"] + new_batting["AB"] - new_batting["H"]
new_batting["full_eb"] = new_batting["alpha"] / (new_batting["alpha"] + new_batting["beta"])

In [88]:
new_batting.head()

,playerID,AB,H,yearID,average,name,eb_estimate,alpha_0,beta_0,alpha_1,beta_1,new_eb,bats_B,bats_L,year_1,year_2,year_3,year_1:bats_L,year_1:bats_B,year_2:bats_L,year_2:bats_B,year_3:bats_L,year_3:bats_B,alpha,beta,beta_2,full_eb
1791,cobbty01,11435,4189,1916.500000,0.366331,Ty Cobb,0.362864,155.962532,390.890474,4344.962532,7636.890474,0.362629,0,1,0.385753,0.805572,-0.191325,0.385753,0.0,0.805572,0.0,-0.191325,-0.0,185.094214,430.674688,7676.674688,0.300590
4512,hornsro01,8173,2930,1926.291667,0.358497,Rogers Hornsby,0.354065,153.158919,393.694088,3083.158919,5636.694088,0.353579,0,0,0.275787,0.920190,-0.195976,0.000000,0.0,0.000000,0.0,-0.000000,-0.0,170.805055,444.963847,5687.963847,0.277385
4720,jacksjo01,4981,1772,1914.071429,0.355752,Shoeless Joe Jackson,0.348884,149.024948,397.828058,1921.024948,3606.828058,0.347517,0,1,0.414766,0.772548,-0.187314,0.414766,0.0,0.772548,0.0,-0.187314,-0.0,177.635826,438.133076,3647.133076,0.288478
465,barnero01,2391,860,1875.333333,0.359682,Ross Barnes,0.345865,142.898198,403.954808,1002.898198,1934.954808,0.341371,0,0,0.937880,0.086291,-0.024171,0.000000,0.0,0.000000,0.0,-0.000000,-0.0,158.979334,456.789569,1987.789569,0.258180
2377,delahed01,7510,2597,1895.500000,0.345806,Ed Delahanty,0.341626,152.452674,394.400332,2749.452674,5307.400332,0.341256,0,0,0.654586,0.472539,-0.127126,0.000000,0.0,0.000000,0.0,-0.000000,-0.0,169.855381,445.913521,5358.913521,0.275843
